#####

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
import nltk
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import unidecode
from nltk.stem.rslp import RSLPStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample
from sklearn.neural_network import MLPClassifier
nltk.download('rslp')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
!pip install tensorflow
!pip install keras

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
def tfidvectorizer(ngram,stop_words):
    vect = TfidfVectorizer(ngram_range=ngram,stop_words=stop_words)
    vect.fit(df_train.descricao_reclamacao)
    return vect
def vectorizer(ngram,stop_words):
    vect = CountVectorizer(ngram_range=ngram,stop_words=stop_words)
    vect.fit(df_train.descricao_reclamacao)
    return vect

def rodar_modelo(vect,model):

    X_train = vect.transform(df_train.descricao_reclamacao)
    y_train = df_train["categoria"]

    model.fit(X_train, y_train)

    X_test = vect.transform(df_test.descricao_reclamacao)
    y_prediction = model.predict(X_test)

    # Verificar a accuracia
    y_test = df_test["categoria"]
    accuracy = accuracy_score(y_prediction, y_test)
    print(accuracy)


In [3]:
import pandas as pd

df = pd.read_csv("tickets_reclamacoes_classificados.csv",sep=';')

In [4]:
stopwords = nltk.corpus.stopwords.words('portuguese')

In [5]:
def transformar_categorias():
    global dataset
    label_enconder = LabelEncoder()
    labels_categorias = label_enconder.fit_transform(dataset.categoria)
    dataset.categoria = labels_categorias
def remover_sensura():
    global dataset
    dataset.descricao_reclamacao = dataset.descricao_reclamacao.replace('x', '', regex=True)
def strlower():
    global dataset
    dataset.descricao_reclamacao = dataset.descricao_reclamacao.str.lower()
def retirar_outros():
    global dataset
    dataset = dataset[dataset["categoria"] != "Outros"]
def retirar_acentuacao():
    global dataset
    dataset.descricao_reclamacao = [unidecode.unidecode(texto) for texto in dataset.descricao_reclamacao]
def lematize_stemmer():
    global dataset
    rslp = RSLPStemmer()
    def stemmer(line):
        return ' '.join([rslp.stem(token) for token in line])
    #tokenizar o texto
    dataset.descricao_reclamacao = dataset.descricao_reclamacao.apply(word_tokenize)
    #aplicar lematize
    dataset.descricao_reclamacao = dataset.descricao_reclamacao.apply(stemmer)
    #juntar novamente as palavras
    dataset.descricao_reclamacao = [WordNetLemmatizer().lemmatize(w) for w in dataset.descricao_reclamacao]
def apenas_texto():
    global dataset
    dataset.descricao_reclamacao = dataset.descricao_reclamacao.replace('[^a-zA-Z ]', '', regex=True)
def retirar_espacos():
    dataset.descricao_reclamacao = dataset.descricao_reclamacao.replace(' [^a-zA-Z]', '', regex=True)

In [116]:
#Limpar memoria
import gc
gc.collect()

0

In [99]:
dataset_use = df.copy()[["categoria","descricao_reclamacao"]]

dataset = pd.DataFrame()
numero_amostra = 250
dataset = pd.concat([dataset,resample(dataset_use[dataset_use.categoria == "Cartão de crédito / Cartão pré-pago"],replace=True,n_samples=numero_amostra,random_state=42)])
dataset = pd.concat([dataset,resample(dataset_use[dataset_use.categoria == "Hipotecas / Empréstimos"],replace=True,n_samples=numero_amostra,random_state=42)])
dataset = pd.concat([dataset,resample(dataset_use[dataset_use.categoria == "Outros"],replace=True,n_samples=numero_amostra,random_state=42)])
dataset = pd.concat([dataset,resample(dataset_use[dataset_use.categoria == "Roubo / Relatório de disputa"],replace=True,n_samples=numero_amostra,random_state=42)])
dataset = pd.concat([dataset,resample(dataset_use[dataset_use.categoria == "Serviços de conta bancária"],replace=True,n_samples=numero_amostra,random_state=42)])

#retirar_outros()
transformar_categorias()
retirar_acentuacao()
apenas_texto()
remover_sensura()
strlower()
#retirar_espacos()
#lematize_stemmer()

df_train, df_test = train_test_split(
        dataset,
        test_size = 0.25,
        random_state = 42
)

vecuni = vectorizer((1,1),stopwords)

X_train = vecuni.transform(df_train.descricao_reclamacao)
y_train = df_train["categoria"]

X_train

<937x11357 sparse matrix of type '<class 'numpy.int64'>'
	with 79278 stored elements in Compressed Sparse Row format>

In [100]:
from tensorflow.keras.layers import LSTM,Embedding,SimpleRNN,Dense,GlobalAveragePooling1D,Dropout,Conv1D,MaxPooling1D,GlobalAveragePooling1D,SpatialDropout1D,Bidirectional,BatchNormalization
from tensorflow.keras import Sequential
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(Dense(128,input_dim=X_train.shape[1],activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(5, activation='sigmoid'))

model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

model.fit(X_train.toarray(), to_categorical(y_train, num_classes=5) ,validation_split=0.2, epochs=500, batch_size=16,callbacks=[EarlyStopping(monitor='loss',patience=7, min_delta=0.0001)])
#model.summary()

Epoch 1/500
47/47 [==============================] - 1s 9ms/step - loss: 1.6076 - accuracy: 0.2991 - val_loss: 1.5521 - val_accuracy: 0.4255
Epoch 2/500
47/47 [==============================] - 0s 3ms/step - loss: 1.4569 - accuracy: 0.5741 - val_loss: 1.4089 - val_accuracy: 0.4947
Epoch 3/500
47/47 [==============================] - 0s 3ms/step - loss: 1.1842 - accuracy: 0.7383 - val_loss: 1.1810 - val_accuracy: 0.7394
Epoch 4/500
47/47 [==============================] - 0s 3ms/step - loss: 0.8548 - accuracy: 0.9132 - val_loss: 0.9853 - val_accuracy: 0.7394
Epoch 5/500
47/47 [==============================] - 0s 2ms/step - loss: 0.5710 - accuracy: 0.9800 - val_loss: 0.8269 - val_accuracy: 0.7660
Epoch 6/500
47/47 [==============================] - 0s 2ms/step - loss: 0.3734 - accuracy: 0.9920 - val_loss: 0.7399 - val_accuracy: 0.7553
Epoch 7/500
47/47 [==============================] - 0s 2ms/step - loss: 0.2466 - accuracy: 0.9947 - val_loss: 0.6708 - val_accuracy: 0.7606
Epoch 8/500
4

In [101]:

X_test = vecuni.transform(df_test.descricao_reclamacao)
y_prediction = model.predict(X_test.toarray())
y_prediction = [np.argmax(linha) for linha in y_prediction]

# Verificar a accuracia
y_test = df_test["categoria"]
accuracy = accuracy_score(y_prediction, y_test)
print(accuracy)

10/10 [==============================] - 0s 1ms/step
0.8178913738019169


In [10]:
model = Sequential()

model.add(Embedding(7000, 32 ,input_length = X_train.shape[1]))
model.add(Bidirectional(LSTM(20, return_sequences=True)))
model.add(Bidirectional(LSTM(20)))
model.add(Dense(5,activation='softmax'))

model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

model.fit(X_train.toarray(), to_categorical(y_train, num_classes=5) ,validation_split=0.2, epochs=10, batch_size=32)

Epoch 1/10
24/24 [==============================] - 19s 734ms/step - loss: 1.6120 - accuracy: 0.1789 - val_loss: 1.6089 - val_accuracy: 0.2234
Epoch 2/10
24/24 [==============================] - 17s 710ms/step - loss: 1.6111 - accuracy: 0.2056 - val_loss: 1.6096 - val_accuracy: 0.1862
Epoch 3/10
24/24 [==============================] - 17s 713ms/step - loss: 1.6102 - accuracy: 0.1989 - val_loss: 1.6088 - val_accuracy: 0.2234
Epoch 4/10
24/24 [==============================] - 17s 733ms/step - loss: 1.6097 - accuracy: 0.2016 - val_loss: 1.6094 - val_accuracy: 0.1862
Epoch 5/10
24/24 [==============================] - 18s 740ms/step - loss: 1.6102 - accuracy: 0.1936 - val_loss: 1.6095 - val_accuracy: 0.2234
Epoch 6/10
24/24 [==============================] - 17s 715ms/step - loss: 1.6094 - accuracy: 0.1802 - val_loss: 1.6089 - val_accuracy: 0.2234
Epoch 7/10
24/24 [==============================] - 17s 704ms/step - loss: 1.6097 - accuracy: 0.2003 - val_loss: 1.6093 - val_accuracy: 0.1862

In [11]:
y_prediction = model.predict(X_test.toarray())
y_prediction = [np.argmax(linha) for linha in y_prediction]

# Verificar a accuracia
y_test = df_test["categoria"]
accuracy = accuracy_score(y_prediction, y_test)
print(accuracy)

10/10 [==============================] - 3s 268ms/step
0.16932907348242812


##### Vamos tentar um modelo com mais neuronios

In [24]:

model = Sequential()
model.add(Dense(256,input_dim=X_train.shape[1],activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(5, activation='sigmoid'))

model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])
model.fit(X_train.toarray(), to_categorical(y_train, num_classes=5) ,validation_split=0.2, epochs=500, batch_size=32,callbacks=[EarlyStopping(monitor='loss',patience=7, min_delta=0.0001)])

Epoch 1/500
24/24 [==============================] - 1s 17ms/step - loss: 1.6245 - accuracy: 0.1963 - val_loss: 1.6079 - val_accuracy: 0.2128
Epoch 2/500
24/24 [==============================] - 0s 4ms/step - loss: 1.6078 - accuracy: 0.2150 - val_loss: 1.5983 - val_accuracy: 0.2713
Epoch 3/500
24/24 [==============================] - 0s 4ms/step - loss: 1.5789 - accuracy: 0.4059 - val_loss: 1.5581 - val_accuracy: 0.4362
Epoch 4/500
24/24 [==============================] - 0s 4ms/step - loss: 1.4915 - accuracy: 0.5007 - val_loss: 1.4601 - val_accuracy: 0.5000
Epoch 5/500
24/24 [==============================] - 0s 4ms/step - loss: 1.3465 - accuracy: 0.5821 - val_loss: 1.3630 - val_accuracy: 0.5000
Epoch 6/500
24/24 [==============================] - 0s 4ms/step - loss: 1.2051 - accuracy: 0.6061 - val_loss: 1.2745 - val_accuracy: 0.5213
Epoch 7/500
24/24 [==============================] - 0s 4ms/step - loss: 1.0741 - accuracy: 0.7089 - val_loss: 1.1985 - val_accuracy: 0.5904
Epoch 8/500


In [25]:
y_prediction = model.predict(X_test.toarray())
y_prediction = [np.argmax(linha) for linha in y_prediction]

# Verificar a accuracia
y_test = df_test["categoria"]
accuracy = accuracy_score(y_prediction, y_test)
print(accuracy)

10/10 [==============================] - 0s 4ms/step
0.7156549520766773


##### Vamos tentar diminuir os neuronios e adicionar dropout entre as camadas

In [62]:
model = Sequential()
model.add(Dense(64,input_dim=X_train.shape[1],activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(16, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(5, activation='sigmoid'))

model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])
model.fit(X_train.toarray(), to_categorical(y_train, num_classes=5) ,validation_split=0.2, epochs=500, batch_size=32,callbacks=[EarlyStopping(monitor='loss',patience=30, min_delta=0.0001)])

Epoch 1/500
24/24 [==============================] - 1s 16ms/step - loss: 1.7136 - accuracy: 0.1896 - val_loss: 1.6118 - val_accuracy: 0.2021
Epoch 2/500
24/24 [==============================] - 0s 4ms/step - loss: 1.6887 - accuracy: 0.2003 - val_loss: 1.6025 - val_accuracy: 0.2553
Epoch 3/500
24/24 [==============================] - 0s 4ms/step - loss: 1.6698 - accuracy: 0.2203 - val_loss: 1.5972 - val_accuracy: 0.2766
Epoch 4/500
24/24 [==============================] - 0s 4ms/step - loss: 1.6407 - accuracy: 0.2537 - val_loss: 1.5903 - val_accuracy: 0.3723
Epoch 5/500
24/24 [==============================] - 0s 4ms/step - loss: 1.6388 - accuracy: 0.1989 - val_loss: 1.5808 - val_accuracy: 0.4149
Epoch 6/500
24/24 [==============================] - 0s 4ms/step - loss: 1.6144 - accuracy: 0.2550 - val_loss: 1.5668 - val_accuracy: 0.5053
Epoch 7/500
24/24 [==============================] - 0s 4ms/step - loss: 1.5922 - accuracy: 0.2497 - val_loss: 1.5433 - val_accuracy: 0.5585
Epoch 8/500


In [63]:
y_prediction = model.predict(X_test.toarray())
y_prediction = [np.argmax(linha) for linha in y_prediction]

# Verificar a accuracia
y_test = df_test["categoria"]
accuracy = accuracy_score(y_prediction, y_test)
print(accuracy)

10/10 [==============================] - 0s 5ms/step
0.7412140575079872


In [66]:
model = Sequential()

model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation = 'softmax')) # Rectified Linear Unit Activation Function
model.add(Dense(15, activation = 'softmax'))
model.add(Dense(5, activation = 'softmax')) # Softmax for multi-class classification

model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])
model.fit(X_train.toarray(), to_categorical(y_train, num_classes=5) ,validation_split=0.2, epochs=500, batch_size=32,callbacks=[EarlyStopping(monitor='loss',patience=30, min_delta=0.0001)])

Epoch 1/500
24/24 [==============================] - 1s 14ms/step - loss: 1.6113 - accuracy: 0.2310 - val_loss: 1.6110 - val_accuracy: 0.2287
Epoch 2/500
24/24 [==============================] - 0s 11ms/step - loss: 1.6063 - accuracy: 0.2750 - val_loss: 1.6072 - val_accuracy: 0.2447
Epoch 3/500
24/24 [==============================] - 0s 3ms/step - loss: 1.6006 - accuracy: 0.2844 - val_loss: 1.6033 - val_accuracy: 0.2447
Epoch 4/500
24/24 [==============================] - 0s 4ms/step - loss: 1.5944 - accuracy: 0.3338 - val_loss: 1.5986 - val_accuracy: 0.3404
Epoch 5/500
24/24 [==============================] - 0s 3ms/step - loss: 1.5866 - accuracy: 0.3965 - val_loss: 1.5931 - val_accuracy: 0.3830
Epoch 6/500
24/24 [==============================] - 0s 3ms/step - loss: 1.5778 - accuracy: 0.5140 - val_loss: 1.5865 - val_accuracy: 0.5053
Epoch 7/500
24/24 [==============================] - 0s 3ms/step - loss: 1.5672 - accuracy: 0.6315 - val_loss: 1.5790 - val_accuracy: 0.5213
Epoch 8/500

In [67]:
y_prediction = model.predict(X_test.toarray())
y_prediction = [np.argmax(linha) for linha in y_prediction]

# Verificar a accuracia
y_test = df_test["categoria"]
accuracy = accuracy_score(y_prediction, y_test)
print(accuracy)

10/10 [==============================] - 0s 4ms/step
0.8115015974440895


In [75]:
model = Sequential()

model.add(Dense(300, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(5, activation = 'softmax')) # Softmax for multi-class classification

model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

model.fit(X_train.toarray(), to_categorical(y_train, num_classes=5) ,validation_split=0.2, epochs=500, batch_size=32)

Epoch 1/500
24/24 [==============================] - 1s 23ms/step - loss: 1.2296 - accuracy: 0.5648 - val_loss: 1.2485 - val_accuracy: 0.6543
Epoch 2/500
24/24 [==============================] - 0s 5ms/step - loss: 0.3261 - accuracy: 0.8972 - val_loss: 1.0734 - val_accuracy: 0.7181
Epoch 3/500
24/24 [==============================] - 0s 4ms/step - loss: 0.1524 - accuracy: 0.9613 - val_loss: 0.9861 - val_accuracy: 0.7021
Epoch 4/500
24/24 [==============================] - 0s 4ms/step - loss: 0.0670 - accuracy: 0.9947 - val_loss: 0.9072 - val_accuracy: 0.7021
Epoch 5/500
24/24 [==============================] - 0s 4ms/step - loss: 0.0417 - accuracy: 0.9920 - val_loss: 0.8386 - val_accuracy: 0.7234
Epoch 6/500
24/24 [==============================] - 0s 4ms/step - loss: 0.0342 - accuracy: 0.9933 - val_loss: 0.7730 - val_accuracy: 0.7234
Epoch 7/500
24/24 [==============================] - 0s 5ms/step - loss: 0.0262 - accuracy: 0.9987 - val_loss: 0.7245 - val_accuracy: 0.7553
Epoch 8/500


In [76]:
y_prediction = model.predict(X_test.toarray())
y_prediction = [np.argmax(linha) for linha in y_prediction]

# Verificar a accuracia
y_test = df_test["categoria"]
accuracy = accuracy_score(y_prediction, y_test)
print(accuracy)

10/10 [==============================] - 0s 7ms/step
0.7987220447284346


In [92]:
model = Sequential()

model.add(Dense(1024, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.8))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))
model.add(BatchNormalization())
model.add(Dense(5, activation = 'softmax')) # Softmax for multi-class classification


model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

model.fit(X_train.toarray(), to_categorical(y_train, num_classes=5) ,validation_split=0.2, epochs=500, batch_size=32)

Epoch 1/500
24/24 [==============================] - 1s 22ms/step - loss: 2.0306 - accuracy: 0.2497 - val_loss: 1.3581 - val_accuracy: 0.5213
Epoch 2/500
24/24 [==============================] - 0s 6ms/step - loss: 1.6588 - accuracy: 0.3765 - val_loss: 1.1874 - val_accuracy: 0.6383
Epoch 3/500
24/24 [==============================] - 0s 6ms/step - loss: 1.3670 - accuracy: 0.4713 - val_loss: 1.0239 - val_accuracy: 0.7181
Epoch 4/500
24/24 [==============================] - 0s 5ms/step - loss: 1.1728 - accuracy: 0.5287 - val_loss: 0.8812 - val_accuracy: 0.7766
Epoch 5/500
24/24 [==============================] - 0s 6ms/step - loss: 0.9758 - accuracy: 0.6115 - val_loss: 0.7933 - val_accuracy: 0.7872
Epoch 6/500
24/24 [==============================] - 0s 6ms/step - loss: 0.8964 - accuracy: 0.6809 - val_loss: 0.7356 - val_accuracy: 0.7979
Epoch 7/500
24/24 [==============================] - 0s 6ms/step - loss: 0.7778 - accuracy: 0.6969 - val_loss: 0.7012 - val_accuracy: 0.7979
Epoch 8/500


In [93]:
y_prediction = model.predict(X_test.toarray())
y_prediction = [np.argmax(linha) for linha in y_prediction]

# Verificar a accuracia
y_test = df_test["categoria"]
accuracy = accuracy_score(y_prediction, y_test)
print(accuracy)

10/10 [==============================] - 0s 2ms/step
0.7284345047923323


##### Agora que fiz uns testes, vou tentar executar com a base completa

In [111]:
dataset_use = df.copy()[["categoria","descricao_reclamacao"]]

dataset = pd.DataFrame()
numero_amostra = 2000
dataset = pd.concat([dataset,resample(dataset_use[dataset_use.categoria == "Cartão de crédito / Cartão pré-pago"],replace=True,n_samples=numero_amostra,random_state=42)])
dataset = pd.concat([dataset,resample(dataset_use[dataset_use.categoria == "Hipotecas / Empréstimos"],replace=True,n_samples=numero_amostra,random_state=42)])
dataset = pd.concat([dataset,resample(dataset_use[dataset_use.categoria == "Outros"],replace=True,n_samples=numero_amostra,random_state=42)])
dataset = pd.concat([dataset,resample(dataset_use[dataset_use.categoria == "Roubo / Relatório de disputa"],replace=True,n_samples=numero_amostra,random_state=42)])
dataset = pd.concat([dataset,resample(dataset_use[dataset_use.categoria == "Serviços de conta bancária"],replace=True,n_samples=numero_amostra,random_state=42)])

#retirar_outros()
transformar_categorias()
retirar_acentuacao()
apenas_texto()
remover_sensura()
strlower()
#retirar_espacos()
#lematize_stemmer()

df_train, df_test = train_test_split(
        dataset,
        test_size = 0.25,
        random_state = 42
)
def tfidvectorizer(ngram,stop_words = []):
    vect = TfidfVectorizer(ngram_range=ngram,stop_words=stop_words)
    vect.fit(df_train.descricao_reclamacao)
    return vect
def vectorizer(ngram,stop_words):
    vect = CountVectorizer(ngram_range=ngram,stop_words=stop_words)
    vect.fit(df_train.descricao_reclamacao)
    return vect
vecuni = vectorizer((1,1),stopwords)

X_train = vecuni.transform(df_train.descricao_reclamacao)
y_train = df_train["categoria"]

X_train

<7500x32601 sparse matrix of type '<class 'numpy.int64'>'
	with 652324 stored elements in Compressed Sparse Row format>

In [112]:
model = Sequential()
model.add(Dense(128,input_dim=X_train.shape[1],activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(5, activation='sigmoid'))

model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

model.fit(X_train.toarray(), to_categorical(y_train, num_classes=5) ,validation_split=0.2, epochs=500, batch_size=32,callbacks=[EarlyStopping(monitor='loss',patience=7, min_delta=0.0001)])
#model.summary()

Epoch 1/500
188/188 [==============================] - 1s 6ms/step - loss: 1.2962 - accuracy: 0.5985 - val_loss: 0.7982 - val_accuracy: 0.8340
Epoch 2/500
188/188 [==============================] - 1s 5ms/step - loss: 0.4355 - accuracy: 0.9318 - val_loss: 0.3625 - val_accuracy: 0.9067
Epoch 3/500
188/188 [==============================] - 1s 5ms/step - loss: 0.1450 - accuracy: 0.9818 - val_loss: 0.2888 - val_accuracy: 0.9120
Epoch 4/500
188/188 [==============================] - 1s 5ms/step - loss: 0.0624 - accuracy: 0.9963 - val_loss: 0.2647 - val_accuracy: 0.9173
Epoch 5/500
188/188 [==============================] - 1s 5ms/step - loss: 0.0335 - accuracy: 0.9988 - val_loss: 0.2651 - val_accuracy: 0.9120
Epoch 6/500
188/188 [==============================] - 1s 5ms/step - loss: 0.0206 - accuracy: 0.9997 - val_loss: 0.2661 - val_accuracy: 0.9167
Epoch 7/500
188/188 [==============================] - 1s 5ms/step - loss: 0.0134 - accuracy: 1.0000 - val_loss: 0.2699 - val_accuracy: 0.9167

In [114]:
X_test = vecuni.transform(df_test.descricao_reclamacao)
y_prediction = model.predict(X_test.toarray())
y_prediction = [np.argmax(linha) for linha in y_prediction]

# Verificar a accuracia
y_test = df_test["categoria"]
accuracy = accuracy_score(y_prediction, y_test)
print(accuracy)

79/79 [==============================] - 0s 4ms/step
0.902
